# ascii_art_generator

## importセクション

In [ ]:
import os

import IPython.display
import emoji
import numpy
import zenhan
from PIL import Image, ImageDraw, ImageFont, ImageOps, ImageFilter


## パラメータセクション

パラメータをセットしてください。

In [2]:
# 画像のパス
image_path = 'input.jpg'

# アスキーアートに使用する文字一覧 (全角)
# (デフォルトで月の絵文字が追加される)
ascii_chars = set()

# 文字用フォントのファイル名
ascii_char_font_file_name = 'Symbola_hint'

# アスキーアート用フォントの大きさ
ascii_art_font_size = 20

# アスキーアートの大きさを決めるために必要なパラメータ
ascii_art_size_params = {
    # TODO: 最大の大きさを元に大きさを調整する場合、以下に最大の大きさ (横 * 縦) を代入し、コメントアウトを解除
    'max size': numpy.array([150, 150])
    # TODO: 最大文字数を元に大きさを調整する場合、以下に最大文字数を代入し、コメントアウトを解除
    # 'max str len': 140
}


## 処理セクション

In [5]:
# 月の絵文字をアスキーアートに使用する文字として追加
for c in (':new_moon:', ':waxing_crescent_moon:', ':first_quarter_moon:', ':waxing_gibbous_moon:',
          ':waning_crescent_moon:', ':last_quarter_moon:', ':waning_gibbous_moon:', ':full_moon:'):
    ascii_chars.add(emoji.emojize(c))

# アスキーアートに使用する文字を全角化
ascii_chars = {zenhan.h2z(c) for c in ascii_chars}


In [6]:
# 文字用フォント
# ascii_fonts['ascii']: アスキーアート化の際に使用
# ascii_fonts['image']: 出力の際に使用
ascii_fonts = {'ascii': ImageFont.truetype(ascii_char_font_file_name),
               'image': ImageFont.truetype(ascii_char_font_file_name, ascii_art_font_size)}

# 文字一覧
# ascii_images['ascii']: アスキーアート化の際に使用
# ascii_images['image']: 出力の際に使用
ascii_images = {'ascii': dict(), 'image': dict()}

# 文字の大きさ
# ascii_size['ascii']['size']: アスキーアート化の際に使用する画像の大きさ (横 * 縦)
# ascii_size['ascii']['shape']: アスキーアート化の際に使用する行列の大きさ (縦 * 横)
# ascii_size['image']['size']: 出力の際に使用する画像の大きさ (横 * 縦)
# ascii_size['image']['shape']: 出力の際に使用する行列の大きさ (縦 * 横)
ascii_size = {'ascii': dict(), 'image': dict()}

# 文字をグレースケール化
for kind in ascii_images.keys():
    for ascii_char in ascii_chars:
        ascii_char_image = Image.new('L', ascii_fonts[kind].getsize(ascii_char))
        ImageDraw.Draw(ascii_char_image).text((0, 0), ascii_char, 'white', ascii_fonts[kind])
        ascii_images[kind][ascii_char] = ImageOps.invert(ascii_char_image.crop(ascii_char_image.getbbox()))

    ascii_size_ = numpy.array([mi.size for mi in ascii_images[kind].values()]).min()
    ascii_size[kind]['size'] = numpy.array([ascii_size_ for _ in range(2)])
    ascii_size[kind]['shape'] = ascii_size[kind]['size'][::-1]

    for ascii_char in ascii_images[kind].keys():
        # 文字の大きさを揃える
        ascii_images[kind][ascii_char] = ascii_images[kind][ascii_char].resize(ascii_size[kind]['size'],
                                                                               Image.ANTIALIAS)

        # 文字を行列化
        if kind == 'ascii':
            ascii_images[kind][ascii_char] = numpy.array(ascii_images[kind][ascii_char])


In [6]:
# アスキーアートの大きさ (横 * 縦)
ascii_art_size = numpy.array(image['base'].size) / ascii_size['ascii']['size']
if 'max size' in ascii_art_size_params:
    argmax_image_size_ = numpy.argmax(image['base'].size)
    max_ascii_art_size_ = ascii_art_size_params['max size'][argmax_image_size_]
    image_size_ = numpy.max(image['base'].size)
    if max_ascii_art_size_ * ascii_size['ascii']['size'][argmax_image_size_] < image_size_:
        ascii_art_size = max_ascii_art_size_ * numpy.array(image['base'].size) / image_size_
elif 'max str len' in ascii_art_size_params:
    if ascii_art_size_params['max str len'] * ascii_size['ascii']['size'].prod() < numpy.prod(image['image'].size):
        ascii_art_size = numpy.sqrt(ascii_art_size_params['max str len'] * numpy.array(image['image'].size)
                                    / numpy.array(image['image'].size[::-1]))

# アスキーアートの大きさの小数点以下切り捨て
ascii_art_size = ascii_art_size.astype(numpy.int64)


In [34]:
# 画像
# image['base']: ベースとなる画像
# image['image']: アスキーアート化する画像 (ベースとなる画像を加工)
# image['ascii']: 行列化した画像
image = {'base': Image.open(image_path)}

# 画像の余白を切り抜き
image['image'] = image['base'].crop(numpy.array(image['base'].getbbox()))

# 画像の透過部を白で塗色
if image['image'].mode == 'RGBA' or 'transparency' in image['image'].info:
    image['image'] = Image.alpha_composite(Image.new(image['image'].mode, image['image'].size, 'white'),
                                           image['image'])

# 画像の輪郭を強調
image['image'] = image['image'].filter(ImageFilter.UnsharpMask(10, 200, 5))

# 画像をグレースケール化
image['image'] = image['image'].convert('L')

# 画像をリサイズ
image['image'] = image['image'].resize(ascii_art_size * ascii_size['ascii']['size'], Image.ANTIALIAS)

# 画像を行列化
image['ascii'] = numpy.array(image['image'])


In [7]:
# アスキーアート
# ascii_art['ascii']: 文字で表されるアスキーアート
# ascii_art['image']['image']: 画像で表されるアスキーアートのImageオブジェクト
# ascii_art['image']['draw']: 画像で表されるアスキーアートのImageDrawオブジェクト
ascii_art = {'ascii': list(), 'image': dict()}
ascii_art['image']['image'] = Image.new('L', tuple(ascii_art_size * ascii_size['image']['size']), 'white')
ascii_art['image']['draw'] = ImageDraw.Draw(ascii_art['image']['image'])

# 画像をアスキーアート化
for i in range(image['ascii'].shape[0] // ascii_size['ascii']['shape'][0]):
    ascii_art['ascii'].append('')
    for j in range(image['ascii'].shape[1] // ascii_size['ascii']['shape'][1]):
        # 画像の一部
        part_image = image['ascii'][ascii_size['ascii']['shape'][0] * i
                                    :ascii_size['ascii']['shape'][0] * (i + 1),
                     ascii_size['ascii']['shape'][1] * j
                     :ascii_size['ascii']['shape'][1] * (j + 1)]

        # 最大類似度
        max_similarity = None

        # 画像の一部ともっとも類似している文字
        max_ascii_char = None

        # 画像の一部ともっとも類似する文字を導出
        for ascii_char, ascii_char_image in ascii_images['ascii'].items():
            # 類似度
            similarity = numpy.sum(part_image * ascii_char_image)

            if max_similarity is None or max_similarity < similarity:
                max_similarity = similarity
                max_ascii_char = ascii_char

        # 画像の一部ともっとも類似している文字をアスキーアートに追加
        ascii_art['ascii'][-1] += max_ascii_char
        ascii_art['image']['draw'].bitmap((ascii_size['image']['size'][0] * j,
                                           ascii_size['image']['size'][1] * i),
                                          ascii_images['image'][max_ascii_char])


## 表示セクション

### 入力

#### アスキーアートに使用する文字

In [8]:
IPython.display.display(IPython.display.HTML('<b>【{}種類】</b>'.format(len(ascii_chars))))
print('  '.join(ascii_chars))


<b>【8種類】</b>

🌑  🌖  🌕  🌒  🌓  🌗  🌔  🌘


#### 入力画像

In [9]:
IPython.display.display(IPython.display.HTML('<b>【{} * {}】</b>'.format(*image['base'].size)))
image['base']


### 中間表現

#### アスキーアート化する画像

In [10]:
IPython.display.display(IPython.display.HTML('<b>【{} * {}】</b>'.format(*image['image'].size)))
image['image']

### 出力

#### アスキーアート (文字)

In [11]:
IPython.display.display(IPython.display.HTML('<b>【{} * {}】</b>'.format(*ascii_art_size)))
print(os.linesep.join(ascii_art['ascii']))


#### アスキーアート (画像)

In [12]:
IPython.display.display(IPython.display.HTML('<b>【{} * {}】</b>'.format(*ascii_art['image']['image'].size)))
ImageOps.invert(ascii_art['image']['image'].crop(ascii_art['image']['image'].getbbox()))
